# Versão 8 dos testes realizados para o modelo


In [1]:
import numpy as np
import pandas as pd 

import matplotlib as mpl 
import matplotlib.cm as cm 
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import _stop_words
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer 

import string
import re

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report
from sklearn import metrics

from time import time

import warnings
warnings.filterwarnings("ignore")

from pathlib import Path

In [2]:
# Lê as propostas com o Pandas
propostas = pd.read_csv("../../production/data_extraction/propostas.csv")

propostas  # Impressão do DataFrame de propostas

,Categoria,Texto
0,Turismo,Turismo: esse é o Destino. <p><strong>Objetivo...
1,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia. <p><stron...
2,Agricultura e Pecuária,Agropecuária Sustentável. <p>Objetivo: Contrib...
3,Saúde,Atenção Primária à Saúde. <p>Fortalecer a Aten...
4,Saúde,Atenção Especializada à Saúde. <p>Ampliar o ac...
...,...,...
8431,Saúde,Microchipagem de animais de companhia. Microch...
8432,Secretaria Geral da Presidência da República,"Direito a atendimento eficiente, é um direito..."
8433,Previdência Social,Rever Reforma da Previdência. Rever a reforma ...
8434,Meio Ambiente e Mudança do Clima,Preservação da Serra do Espinhaço. A Serra do ...


In [3]:
# Expressões regulares para remoção de poluição de dados nos textos 
# REGX_URL = r"https?://[A-Za-z0-9./\-]+" # Regex for URLs
REGX_HTML = r"<[^<]+?>" # Regex for HTML tags
REGX_ENDING = r'Órgão Responsável:.+' # Regex for the part of the text

In [4]:
# Retirando Tags HTML e URLS do texto
def preprocessing(text):
  # text = text.lower()

  text = re.sub(REGX_HTML, '', text)  # Removendo tags HTML
  # text = re.sub(REGX_URL, '', text) # Revomendo URLs
  text = re.sub(REGX_ENDING, '', text)

  # tokens = [t.lemma_ for t in nlp(text) if t not in STOP_WORDS and not t.is_punct]

  return text

#  Ajustando textos da coluna Corpo
propostas['Texto'] = propostas['Texto'].apply(preprocessing)
propostas.head()

,Categoria,Texto
0,Turismo,Turismo: esse é o Destino. Objetivo: Posiciona...
1,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia. Objetivo:...
2,Agricultura e Pecuária,Agropecuária Sustentável. Objetivo: Contribuir...
3,Saúde,Atenção Primária à Saúde. Fortalecer a Atenção...
4,Saúde,Atenção Especializada à Saúde. Ampliar o acess...


In [5]:
print(propostas.Categoria.value_counts())

Categoria
Saúde                                                             1368
Educação                                                          1261
Direitos Humanos e Cidadania                                       528
Meio Ambiente e Mudança do Clima                                   497
Transportes                                                        473
Desenvolvimento e Assistência Social, Família e Combate à Fome     418
Trabalho e Emprego                                                 377
Justiça e Segurança Pública                                        342
Cidades                                                            294
Gestão e Inovação em Serviços Públicos                             284
Desenvolvimento Agrário e Agricultura Familiar                     255
Previdência Social                                                 249
Cultura                                                            224
Mulheres                                                           

# Realizando a criacao de novas propostas e verificando a performance no modelo



In [6]:
# Lê as propostas com o Pandas

simuladas = pd.read_csv("../../production/simulacao/propostas_simuladas.csv")
simuladas  # Impressão do DataFrame de propostas

,Categoria,Texto
0,Casa Civil,É ótimo ver a conscientização sobre igualdade ...
1,Casa Civil,Tornar a Casa Civil mais ecológica e sustentáv...
2,Casa Civil,Criar um programa de parcerias com startups na...
3,Casa Civil,Vamos estabelecer metas ambiciosas para reduzi...
4,Casa Civil,Vamos criar um comitê de ética interno na Casa...
...,...,...
366,Comunicações,O serviço postal desempenha um papel essencial...
367,Comunicações,A imprensa local desempenha um papel vital na ...
368,Comunicações,Fusões e aquisições no setor de mídia podem te...
369,Comunicações,"Para capacitar os cidadãos na era digital, é c..."


In [7]:
# Juntando os dataframes

propostas_simuladas = pd.DataFrame(simuladas)
propostas = pd.concat([propostas,propostas_simuladas],ignore_index=True)

# Analisando Categorias sem performance


In [8]:
# Escolha a categoria que deseja manter
categoria_desejada = 'Relações Exteriores'

# Filtrando as linhas com a categoria desejada
rel_exteriores = propostas.loc[propostas['Categoria'] == categoria_desejada]

rel_exteriores

,Categoria,Texto
173,Relações Exteriores,Passaporte mais rapido e menos burocratico . S...
1158,Relações Exteriores,Aprimoramento da plataforma Concórdia para fin...
2947,Relações Exteriores,Fortalecimento do Mercosul e do BRICS através...
2992,Relações Exteriores,"Entretenimento . Promover curtas, longas e sér..."
3709,Relações Exteriores,Criação de bolsas de aperfeiçoamento profissio...
4447,Relações Exteriores,Cooperação internacional para combater o tráfi...
6156,Relações Exteriores,Taxação para produção e exportação de alimento...
6536,Relações Exteriores,Criação do Conselho Nacional de Política Exter...
7035,Relações Exteriores,"Estruturação dos escritórios, diretorias e coo..."
7326,Relações Exteriores,Implementação de uma política externa feminist...


In [9]:
rel_exteriores['Texto']

173     Passaporte mais rapido e menos burocratico . S...
1158    Aprimoramento da plataforma Concórdia para fin...
2947     Fortalecimento do Mercosul e do BRICS através...
2992    Entretenimento . Promover curtas, longas e sér...
3709    Criação de bolsas de aperfeiçoamento profissio...
4447    Cooperação internacional para combater o tráfi...
6156    Taxação para produção e exportação de alimento...
6536    Criação do Conselho Nacional de Política Exter...
7035    Estruturação dos escritórios, diretorias e coo...
7326    Implementação de uma política externa feminist...
7444    Fomentação de Estudos no Exterior . Criação de...
8053    Valorização da imagem internacional do Brasil....
8476    Fortalecimento de Alianças Estratégicas: O gov...
8477    Diversificação de Parceiros Comerciais: É impo...
8478    Diplomacia na Área Ambiental: Dada a importânc...
8479    Cooperação em Saúde Global: Em vista da experi...
8480    Promoção dos Direitos Humanos: O Brasil deve m...
8481    Capaci

In [10]:
rel_exteriores['Texto'][173]

'Passaporte mais rapido e menos burocratico . Se exige muitos documentos para renovar o passaporte, isso só sobrecarrega as embaixadas que as vezes demoram muito para emitir um novo passaporte, ou outros serviços, sobretudo quando a pessoa morra fora do Brasil, muitas pessoas reclamam da quantidade de papeis para renovar um passaporte que na logica e comparado com outros países que só pedem a taxa para emissão do passaporte, 2 fotos e comprovante de identidade, com a modernização das tecnologias o MRE deveria emitir passaportes mais rapido'

In [11]:
print(propostas.Categoria.value_counts())

Categoria
Saúde                                                             1368
Educação                                                          1261
Direitos Humanos e Cidadania                                       528
Meio Ambiente e Mudança do Clima                                   497
Transportes                                                        473
Desenvolvimento e Assistência Social, Família e Combate à Fome     418
Trabalho e Emprego                                                 377
Justiça e Segurança Pública                                        342
Cidades                                                            294
Gestão e Inovação em Serviços Públicos                             284
Desenvolvimento Agrário e Agricultura Familiar                     255
Previdência Social                                                 249
Cultura                                                            224
Mulheres                                                           

In [12]:
fig = go.Figure([go.Bar(x=propostas['Categoria'].value_counts().index, y=propostas['Categoria'].value_counts().tolist())])
fig.update_layout(
    title="Quantidade de propostas",
    xaxis_title="Propostas",
    yaxis_title="quantidade")
fig.show()

# Pre Processamento


In [13]:
#Criando índices para as categorias
cats = propostas['Categoria'].unique() # Pegando cada categoria única
cats = dict(enumerate(cats, 0)) # Convertendo para dict (com índices enumerados)
cats = {v:k for k,v in cats.items()}  # Trocando chaves e valores

propostas['id_cats'] = propostas['Categoria'].map(cats) # Inserindo coluna de índices das cats
propostas.head()

,Categoria,Texto,id_cats
0,Turismo,Turismo: esse é o Destino. Objetivo: Posiciona...,0
1,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia. Objetivo:...,1
2,Agricultura e Pecuária,Agropecuária Sustentável. Objetivo: Contribuir...,2
3,Saúde,Atenção Primária à Saúde. Fortalecer a Atenção...,3
4,Saúde,Atenção Especializada à Saúde. Ampliar o acess...,3


In [14]:
propostas['Texto'][0]

'Turismo: esse é o Destino. Objetivo: Posicionar o turismo como vetor de desenvolvimento sustentável e aumentar a competitividade dos destinos e produtos turísticos brasileiros, democratizando o acesso à atividade turística aos cidadãos brasileiros.'

In [15]:
import nltk
import spacy
nlp = spacy.load('pt_core_news_sm')  

nltk.download('stopwords')
nltk.download('wordnet')

stopwords = nltk.corpus.stopwords.words('portuguese')

lemmatizer = WordNetLemmatizer()

def clean(doc):
    
    if isinstance(doc, float) and np.isnan(doc):
        return ''
    
    text_no_namedentities = []
    document = nlp(doc)
    ents = [e.text for e in document.ents]
    for item in document:
        if item.text in ents:
            pass
        else:
            text_no_namedentities.append(item.text)
    doc = (" ".join(text_no_namedentities))

    doc = doc.lower().strip()
    doc = doc.replace("</br>", " ") 
    doc = doc.replace("-", " ") 
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    doc = " ".join([token for token in doc.split() if token not in stopwords])    
    doc = "".join([lemmatizer.lemmatize(word) for word in doc])
    return doc


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/leandro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/leandro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
clean(propostas['Texto'][0])

'turismo objetivo posicionar turismo vetor desenvolvimento sustentável aumentar competitividade destinos produtos turísticos brasileiros democratizando acesso atividade turística cidadãos brasileiros'

In [17]:
# propostas['Texto'] = propostas['Texto'].apply(clean)
propostas.head()

,Categoria,Texto,id_cats
0,Turismo,Turismo: esse é o Destino. Objetivo: Posiciona...,0
1,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia. Objetivo:...,1
2,Agricultura e Pecuária,Agropecuária Sustentável. Objetivo: Contribuir...,2
3,Saúde,Atenção Primária à Saúde. Fortalecer a Atenção...,3
4,Saúde,Atenção Especializada à Saúde. Ampliar o acess...,3


## TF-IDF Vectorizer

![TF-IDF](https://miro.medium.com/max/3136/1*ruCawEw0--m2SeHmAQooJQ.jpeg)

In [18]:
docs = list(propostas['Texto'])
tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_features = 20000) 
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(docs)
docs = tfidf_vectorizer_vectors.toarray()

In [19]:
X = docs 
y = propostas['id_cats']
print(X.shape, y.shape)


(8807, 20000) (8807,)


In [20]:
fig = go.Figure([go.Bar(x=y.value_counts().index, y=y.value_counts().tolist())])
fig.update_layout(
    title="Quantidade de propostas",
    xaxis_title="Propostas",
    yaxis_title="quantidade")
fig.show()

# Train test Split

In [21]:
SEED=123
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(7045, 20000) (7045,)
(1762, 20000) (1762,)


In [22]:
target_names = [
    'Saúde', 'Educação', 'Direitos Humanos e Cidadania', 'Meio Ambiente e Mudança do Clima',
    'Transportes', 'Desenvolvimento e Assistência Social, Família e Combate à Fome',
    'Trabalho e Emprego', 'Justiça e Segurança Pública', 'Cidades',
    'Gestão e Inovação em Serviços Públicos', 'Desenvolvimento Agrário e Agricultura Familiar',
    'Previdência Social', 'Cultura', 'Mulheres', 'Ciência, Tecnologia e Inovação',
    'Fazenda', 'Planejamento e Orçamento', 'Turismo',
    'Desenvolvimento, Indústria, Comércio e Serviços', 'Integração e Desenvolvimento Regional',
    'Agricultura e Pecuária', 'Minas e Energia', 'Igualdade Racial',
    'Secretaria Geral da Presidência da República', 'Esporte', 'Casa Civil',
    'Defesa', 'Comunicações', 'Comunicação Social', 'Relações Institucionais',
    'Povos Indígenas', 'Pesca e Aquicultura', 'Controladoria-Geral da União',
    'Outros', 'Segurança Institucional', 'Banco Central', 'Portos e Aeroportos',
    'Advocacia-Geral da União', 'Relações Exteriores'
]

## Balanceando dados

## Classificação com Naive Bayes 

### Gaussian Naive Bayes


In [23]:
gnb = GaussianNB() 
%time gnb.fit(X_train, y_train)

y_pred_train = gnb.predict(X_train)
y_pred_test = gnb.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))

CPU times: user 1.12 s, sys: 327 ms, total: 1.45 s
Wall time: 1.45 s

Training Accuracy score: 0.9518807665010646
Testing Accuracy score: 0.3444948921679909


In [24]:
print(classification_report(y_test, y_pred_test,target_names=target_names))

                                                                precision    recall  f1-score   support

                                                         Saúde       0.40      0.08      0.14        24
                                                      Educação       0.48      0.25      0.33        51
                                  Direitos Humanos e Cidadania       0.54      0.33      0.41        21
                              Meio Ambiente e Mudança do Clima       0.37      0.62      0.46       274
                                                   Transportes       0.67      0.15      0.25        13
Desenvolvimento e Assistência Social, Família e Combate à Fome       0.27      0.18      0.21        84
                                            Trabalho e Emprego       0.60      0.15      0.24        20
                                   Justiça e Segurança Pública       0.16      0.09      0.12        75
                                                       Cidades 

### Multinomial Naive Bayes

In [25]:
mnb = MultinomialNB() 
%time mnb.fit(X_train, y_train)

y_pred_train = mnb.predict(X_train)
y_pred_test = mnb.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))

CPU times: user 1.33 s, sys: 286 ms, total: 1.61 s
Wall time: 353 ms

Training Accuracy score: 0.34591909155429384
Testing Accuracy score: 0.3166855845629966


In [26]:
print(classification_report(y_test, y_pred_test,target_names=target_names))

                                                                precision    recall  f1-score   support

                                                         Saúde       0.00      0.00      0.00        24
                                                      Educação       0.80      0.08      0.14        51
                                  Direitos Humanos e Cidadania       0.00      0.00      0.00        21
                              Meio Ambiente e Mudança do Clima       0.33      0.92      0.49       274
                                                   Transportes       0.00      0.00      0.00        13
Desenvolvimento e Assistência Social, Família e Combate à Fome       0.00      0.00      0.00        84
                                            Trabalho e Emprego       0.00      0.00      0.00        20
                                   Justiça e Segurança Pública       0.00      0.00      0.00        75
                                                       Cidades 

## Logistic Regression Classifier

In [27]:
lr = LogisticRegression(random_state=SEED)
%time lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))

CPU times: user 6min 39s, sys: 3min 37s, total: 10min 17s
Wall time: 1min 22s

Training Accuracy score: 0.71611071682044
Testing Accuracy score: 0.5488081725312145


In [28]:
print(classification_report(y_test, y_pred_test,target_names=target_names))

                                                                precision    recall  f1-score   support

                                                         Saúde       0.92      0.50      0.65        24
                                                      Educação       0.60      0.69      0.64        51
                                  Direitos Humanos e Cidadania       1.00      0.10      0.17        21
                              Meio Ambiente e Mudança do Clima       0.51      0.87      0.65       274
                                                   Transportes       0.00      0.00      0.00        13
Desenvolvimento e Assistência Social, Família e Combate à Fome       0.42      0.27      0.33        84
                                            Trabalho e Emprego       1.00      0.35      0.52        20
                                   Justiça e Segurança Pública       0.49      0.45      0.47        75
                                                       Cidades 

## Support Vector Machines

In [29]:
svc =  LinearSVC(class_weight='balanced') 
%time svc.fit(X_train, y_train)

y_pred_train = svc.predict(X_train)
y_pred_test = svc.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))

CPU times: user 1.1 s, sys: 278 ms, total: 1.38 s
Wall time: 1.05 s

Training Accuracy score: 0.9761533002129169
Testing Accuracy score: 0.5811577752553916


In [30]:
print(classification_report(y_test, y_pred_test,target_names=target_names))

                                                                precision    recall  f1-score   support

                                                         Saúde       0.73      0.67      0.70        24
                                                      Educação       0.52      0.61      0.56        51
                                  Direitos Humanos e Cidadania       0.31      0.24      0.27        21
                              Meio Ambiente e Mudança do Clima       0.79      0.73      0.76       274
                                                   Transportes       0.67      0.62      0.64        13
Desenvolvimento e Assistência Social, Família e Combate à Fome       0.46      0.31      0.37        84
                                            Trabalho e Emprego       0.65      0.55      0.59        20
                                   Justiça e Segurança Pública       0.47      0.49      0.48        75
                                                       Cidades 

In [31]:
propostas['Categoria'].value_counts()

Categoria
Saúde                                                             1368
Educação                                                          1261
Direitos Humanos e Cidadania                                       528
Meio Ambiente e Mudança do Clima                                   497
Transportes                                                        473
Desenvolvimento e Assistência Social, Família e Combate à Fome     418
Trabalho e Emprego                                                 377
Justiça e Segurança Pública                                        342
Cidades                                                            294
Gestão e Inovação em Serviços Públicos                             284
Desenvolvimento Agrário e Agricultura Familiar                     255
Previdência Social                                                 249
Cultura                                                            224
Mulheres                                                           

## Decision Tree Classifier

In [32]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=SEED)
%time dt.fit(X_train, y_train)

y_pred_train = dt.predict(X_train)
y_pred_test = dt.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))

CPU times: user 40.1 s, sys: 323 ms, total: 40.5 s
Wall time: 40.2 s

Training Accuracy score: 0.9957416607523066
Testing Accuracy score: 0.40068104426787743


In [33]:
print(classification_report(y_test, y_pred_test,target_names=target_names))

                                                                precision    recall  f1-score   support

                                                         Saúde       0.63      0.50      0.56        24
                                                      Educação       0.27      0.25      0.26        51
                                  Direitos Humanos e Cidadania       0.10      0.14      0.12        21
                              Meio Ambiente e Mudança do Clima       0.60      0.60      0.60       274
                                                   Transportes       0.11      0.08      0.09        13
Desenvolvimento e Assistência Social, Família e Combate à Fome       0.21      0.19      0.20        84
                                            Trabalho e Emprego       0.47      0.35      0.40        20
                                   Justiça e Segurança Pública       0.21      0.19      0.20        75
                                                       Cidades 

## Ensembling

In [34]:
from sklearn.ensemble import VotingClassifier

classifiers = [('Decision Tree', dt),
               ('Logistic Regression', lr),
                ('Naive Bayes', gnb)
              ]
vc = VotingClassifier(estimators=classifiers)
# Fit 'vc' to the traing set and predict test set labels
vc.fit(X_train, y_train)
y_pred_train=vc.predict(X_train)
y_pred_test = vc.predict(X_test)
print("Training Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))

Training Accuracy score: 0.9892122072391767
Testing Accuracy score: 0.49432463110102154


In [35]:
print(classification_report(y_test, y_pred_test,target_names=target_names))

                                                                precision    recall  f1-score   support

                                                         Saúde       0.72      0.54      0.62        24
                                                      Educação       0.43      0.69      0.53        51
                                  Direitos Humanos e Cidadania       0.25      0.24      0.24        21
                              Meio Ambiente e Mudança do Clima       0.42      0.86      0.56       274
                                                   Transportes       0.50      0.08      0.13        13
Desenvolvimento e Assistência Social, Família e Combate à Fome       0.29      0.27      0.28        84
                                            Trabalho e Emprego       0.67      0.40      0.50        20
                                   Justiça e Segurança Pública       0.34      0.31      0.32        75
                                                       Cidades 

In [36]:
predictions = pd.Series(vc.predict(X), name="Categoria")
results = pd.concat([predictions],axis=1)
results.to_csv("novas_propostas_resultado.csv",index=False)

: 

In [ ]:
categorias_interesse = ['Relações Exteriores', 'Pesca e Aquicultura', 'Povos Indígenas', 'Secretaria Geral da Presidência da República']

sem_performance = propostas[propostas['Categoria'].isin(categorias_interesse)]


In [ ]:
# Salvando o dataframe filtrado em um novo CSV
sem_performance.to_csv('sem_performance.csv', index=False)